In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_ratings = pd.read_csv("ml-latest-small/ratings.csv")
df_movies = pd.read_csv("ml-latest-small/movies.csv")

In [5]:
df_ratings = df_ratings.merge(df_movies[['movieId', 'title']],how='left', on='movieId')

In [6]:
df_user_item = df_ratings.pivot_table(index=['userId'], columns=['title'], values='rating')

In [7]:
from scipy import sparse

In [9]:
corr_matrix = df_user_item.corr(method='pearson', min_periods=100) # min_periods отвечает за количество наблюдений чтобы сделать оценку по корреляции

In [10]:
user_ratings = df_user_item.loc[7].dropna()
user_ratings

title
2001: A Space Odyssey (1968)           4.0
A.I. Artificial Intelligence (2001)    4.5
Aladdin (1992)                         3.0
American Beauty (1999)                 4.0
Apocalypse Now (1979)                  4.0
                                      ... 
What Women Want (2000)                 4.0
Wild Wild West (1999)                  1.5
X-Men (2000)                           3.5
X-Men: The Last Stand (2006)           4.0
X2: X-Men United (2003)                4.0
Name: 7, Length: 152, dtype: float64

In [11]:
simCandidates = pd.Series()
for i in range(0, len(user_ratings.index)): # Идем по списку всех фильмов оцененных пользвателем
    sims = corr_matrix[user_ratings.index[i]].dropna() # Извлекаем фильмы, похожие на оцененные данным юзером (1)
    sims = sims.map(lambda x: x * user_ratings[i]) # Умножаем корреляцию на оценку пользователя по фильму
    simCandidates = simCandidates.append(sims) # Добавляем индекс в список сравниваемых кандидатов
simCandidates.sort_values(inplace = True, ascending = False)

<ipython-input-11-09b49e4a514b>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates = pd.Series()


In [13]:
sims = corr_matrix[user_ratings.index[5]].dropna()

In [14]:
sims = sims.map(lambda x: x * user_ratings[5])

In [15]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()
simCandidates.sort_values(inplace = True, ascending = False)

In [16]:
intersection_set = set.intersection(set(simCandidates.index), set(user_ratings.index))
not_watched = list(set(simCandidates.index) - set(user_ratings.index))

In [17]:
filteredSims = simCandidates[not_watched].sort_values(ascending = False)

In [18]:
filteredSims

Matrix, The (1999)                                                                30.707095
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    22.002139
Shawshank Redemption, The (1994)                                                  18.925514
Saving Private Ryan (1998)                                                        16.090221
Braveheart (1995)                                                                 14.532443
Men in Black (a.k.a. MIB) (1997)                                                  12.640371
Fight Club (1999)                                                                 12.045625
Lion King, The (1994)                                                             11.318720
Schindler's List (1993)                                                           11.235903
Speed (1994)                                                                      10.869473
Godfather, The (1972)                                                           